In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.dropout1 = nn.Dropout(0.15)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(8)
        self.dropout2 = nn.Dropout(0.15)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(8)
        self.dropout3 = nn.Dropout(0.15)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout(0.15)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout(0.15)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)
        self.batchnorm6 = nn.BatchNorm2d(16)
        self.dropout6 = nn.Dropout(0.15)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv7 = nn.Conv2d(16, 32, 3)
        self.batchnorm7 = nn.BatchNorm2d(32)
        self.dropout7 = nn.Dropout(0.15)
        self.conv8 = nn.Conv2d(32, 16, 3)
        self.batchnorm8= nn.BatchNorm2d(16)
        self.dropout8 = nn.Dropout(0.15)
        self.conv9 = nn.Conv2d(16, 10, 3)

    def forward(self, x):
        x = self.pool1(self.dropout3(self.batchnorm3(F.relu(self.conv3(self.dropout2(self.batchnorm2(F.relu(self.conv2(self.dropout1(self.batchnorm1(F.relu(self.conv1(x)))))))))))))
        x = self.pool2(self.dropout6(self.batchnorm6(F.relu(self.conv6(self.dropout5(self.batchnorm5(F.relu(self.conv5(self.dropout4(self.batchnorm4(F.relu(self.conv4(x)))))))))))))
        x = F.relu(self.dropout8(self.batchnorm8(self.conv8(self.dropout7(self.batchnorm7(F.relu(self.conv7(x))))))))
#        x = F.relu(self.conv9(x))
        x = self.conv9(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             584
       BatchNorm2d-5            [-1, 8, 28, 28]              16
           Dropout-6            [-1, 8, 28, 28]               0
            Conv2d-7            [-1, 8, 28, 28]             584
       BatchNorm2d-8            [-1, 8, 28, 28]              16
           Dropout-9            [-1, 8, 28, 28]               0
        MaxPool2d-10            [-1, 8, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           1,168
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    print("\n EPOC NUMBER IS :", epoch)
    test(model, device, test_loader)

loss=0.09064608067274094 batch_id=937: 100%|██████████| 938/938 [00:15<00:00, 60.60it/s]


 EPOC NUMBER IS : 1



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0975, Accuracy: 9733/10000 (97.33%)



loss=0.04427775740623474 batch_id=937: 100%|██████████| 938/938 [00:15<00:00, 61.21it/s]

...


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9941/10000 (99.41%)



loss=0.02837410569190979 batch_id=937: 100%|██████████| 938/938 [00:15<00:00, 59.55it/s]


 EPOC NUMBER IS : 19



Test set: Average loss: 0.0178, Accuracy: 9943/10000 (99.43%)

